# PLC siemens + realsense D455 rect depth matching

## Snap7 PLC connection

In [ ]:
#common variables
found_part_num=0

## PLC class import

In [8]:
import sys
sys.path.insert(1, '../python')
import plc

## PLC class debug

In [3]:
import snap7
import sys
import logging
from logging import handlers
from logging.handlers import RotatingFileHandler
import traceback
import time
import threading
from snap7.util import *
import yaml
import os
csd = '../python' #os.path.dirname(os.path.abspath(__file__))
config = yaml.safe_load(open(csd+"/config.yml"))

# TODO found_part_num
#found_part_num = 0
camera_db_num = config['plc']['camera_db_num']
reconnect_timeout = config['plc']['reconnect_timeout']

class PLC(threading.Thread):
    def __init__(self, plc_ip):
        #init
        threading.Thread.__init__(self, args=(), name=plc_ip, kwargs=None)
        self.plc_ip = plc_ip
        self.found_part_num = 0
        self.snapshotReq = False
        self.plc_ip = plc_ip
        self.logger = logging.getLogger("_plc_.client")
        #logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))
        logging.basicConfig(level=config['logger']['level'],
                            handlers=[logging.StreamHandler(sys.stdout),
                                      logging.handlers.RotatingFileHandler(config['logger']['debug_file'],
                                                                           maxBytes=(1048576 * 5),
                                                                           backupCount=7)],
                            format=config['logger']['format'])
        self.snap7client = snap7.client.Client()
        self.connection_ok = False
        self.unreachable_time = 0
        self.connect_postpone = False

    def get_bool(self, db_number, offsetbyte, offsetbit):
        return snap7.util.get_bool(self.db_read(db_number, offsetbyte, 1), 0, offsetbit)

    def set_usint(self, db_number, offsetbyte, tag_value):
        tag_data = bytearray(1)
        snap7.util.set_usint(tag_data, 0, tag_value)
        self.snap7client.db_write(db_number, offsetbyte, tag_data)
        return True

    def db_read(self, db_number, offsetbyte, len_arr):
        return self.snap7client.db_read(db_number, offsetbyte, len_arr)

    def db_write(self, db_number, offsetbyte, tag_data):
        return self.snap7client.db_write(db_number, offsetbyte, tag_data)

    def run(self):
        self.logger.info(f"Connection with PLC {self.plc_ip} started")
        cur_thread = threading.current_thread()
        # Основной цикл
        while getattr(cur_thread, "do_run", True):
            try:
                time.sleep(1.2)
                if self.unreachable_time == 0 or (time.time() - self.unreachable_time) > reconnect_timeout:
                    if not self.snap7client.get_connected():
                        # Подключение к контроллеру ...
                        try:
                            self.connection_ok = False
                            self.logger.info(f"Подключение к контроллеру {self.plc_ip}...")
                            self.snap7client.connect(self.plc_ip, 0, 1)
                        except Exception as error:
                            self.logger.error(f"Не удалось подключиться к контроллеру: {self.plc_ip}\n"
                                              f"Ошибка {str(error)} {traceback.format_exc()}")
                            snap7.client.logger.disabled = True
                            self.unreachable_time = time.time()
                    else:
                        if not self.connection_ok:
                            self.connection_ok = True
                            self.unreachable_time = 0
                            self.logger.info(f"Соединение открыто {self.plc_ip}")
                            snap7.client.logger.disabled = False
                        try:
                            if not self.snapshotReq:
                                self.snapshotReq = self.get_bool(db_number=camera_db_num, offsetbyte=0, offsetbit=0)
                                if self.snapshotReq:
                                    self.logger.info(f"Строб съёмки пришёл {self.snapshotReq}")
                        except Exception as error:
                            self.logger.error(f"Не удалось считать строб съёмки: DB{camera_db_num}.DBX0.0\n"
                                              f"Ошибка {str(error)} {traceback.format_exc()}")
                            self.snap7client.disconnect()


                        if self.snapshotReq:
                            if self.found_part_num > 0:
                               self.logger.info(f"Запись результата распознования - номер найденной детали - {self.found_part_num}")
                               try:
                                   self.set_usint(db_number=camera_db_num, offsetbyte=1, tag_value=self.found_part_num)
                                   self.found_part_num = 0
                               except Exception as error:
                                   self.logger.error(f"Не удалось записать результат съёмки: DB{camera_db_num}.DBB1\n"
                                                     f"Ошибка {str(error)} {traceback.format_exc()}")
                                   self.snap7client.disconnect()
            except Exception as error:
                self.logger.error(f"Не удалось обработать цикл класса plc\n"
                                  f"Ошибка {str(error)} {traceback.format_exc()}")

In [3]:
my_plc = PLC('192.168.1.101')

In [4]:
my_plc.start()

## Part detection with Intel realsense D455 and OpenCV

In [ ]:
scaleAbs_alpha = 0.3
threshold = 0.65


In [1]:
#var huck
found_part_num = 0
teach_part_num = 0

#separate file - configurations of parts and settings
tolerance = 0.05
scaleAbs_alpha = 0.3
threshold = 0.65
#resolution_x,resolution_y=640,480
resolution_x,resolution_y=848,480
#resolution_x,resolution_y=1280,720
fps=5
parts=[ dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ А',h=212,w=89,Zavg=400),
        dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ B',h=208,w=87,Zavg=400),
        dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ C',h=208,w=80.6,Zavg=400),
        dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ D',h=208,w=80,Zavg=400),
        dict(code='8АТ-1250-11',part_name='Корпус маятника',step_name='Установ E',h=528,w=268,Zavg=400),
        dict(code='17115.2900.77',part_name='Переходник',step_name='Установ А',h=273,w=206,Zavg=400),
        dict(code='17115.2900.77',part_name='Переходник',step_name='Установ B',h=209.03,w=206,Zavg=400)]


import pyrealsense2 as rs
import numpy as np
import pyshine as ps
import socket
import cv2
import os

templateDir = 'templates'
lstdr = os.listdir(templateDir)
parts = [f for f in lstdr if os.path.isdir(templateDir+'/'+f)] 
templates = {str(p):[f for f in os.listdir(templateDir+'/'+p) if os.path.isfile(templateDir+'/'+p+'/'+f) and f.endswith('.png')] for p in parts} 

HTML=f"""
<html>
<body>
<center><img src="stream.mjpg" width='{resolution_x*2}' height='{resolution_y}' autoplay playsinline></center>
</body>
</html>
"""

StreamProps = ps.StreamProps
StreamProps.set_Page(StreamProps,HTML)
address = (socket.gethostbyname(socket.gethostname()),9001) # Enter your IP address 


class ImgCapture():

    def __init__(self):
        pass
    
    def read(self):

        # Wait for a coherent pair of frames: depth and color
        self.frames = pipeline.wait_for_frames()
        depth_frame = self.frames.get_depth_frame()
        color_frame = self.frames.get_color_frame()
        """
        # Convert images to numpy arrays
        #depth_image = np.asanyarray(depth_frame.get_data())
        frames = []
        for x in range(15):
            frameset = pipeline.wait_for_frames()
            frames.append(frameset.get_depth_frame())
        
        hole_filling = rs.hole_filling_filter()
        for x in range(15):
            frame = hole_filling.process(frames[x])
            frames[x] = np.asanyarray(frame.get_data())
        depth_image = np.min(frames, axis=0)        
        """
        hole_filling = rs.hole_filling_filter()
        depth_frame = hole_filling.process(depth_frame)
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=scaleAbs_alpha), cv2.COLORMAP_JET)
        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape
        
        #print (f"templates loop for {templates}")
        for part, pics in templates.items():
            #print (f"look for part {part}")
            for file in pics:
                #print (f"look for template {template}")
                template = cv2.imread(f"{templateDir}/{part}/{file}")
                h, w = template.shape[:2]
                methods = [cv2.TM_SQDIFF_NORMED, cv2.TM_CCORR_NORMED, cv2.TM_CCOEFF_NORMED]
                res = cv2.matchTemplate(depth_colormap, template, methods[2])
                loc = np.where(res >= threshold)  # Coordinates y, x where the matching degree is greater than threshold
                #print("loc", loc)
                #for pt in zip(*loc[::-1]):  # * Indicates optional parameters
                if len(loc[0])>0:
                    print (f"Part {part} found" )
                    pt = list(zip(*loc[::-1]))[0]
                    right_bottom = (pt[0] + w, pt[1] + h)
                    cv2.rectangle(depth_colormap, pt, right_bottom, (0, 0, 255), 1)
                    continue
        
        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
        images = np.hstack((depth_colormap, color_image))
        
        if images is not None:
            ret = True
        return(ret, images)

    def isOpened(self):
        ret, _, _ = self.rs.get_frame_stream()
        return(ret)   
        

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
#depth_sensor = device.first_depth_sensor()
device_product_line = str(device.get_info(rs.camera_info.product_line))
print('device_product_line',device_product_line)

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)
align = rs.align(rs.stream.color)
config.enable_stream(rs.stream.depth, resolution_x, resolution_y, rs.format.z16, fps)
config.enable_stream(rs.stream.color, resolution_x, resolution_y, rs.format.bgr8, fps)

# Start streaming
pipeline.start(config)
try:
            StreamProps.set_Mode(StreamProps,'cv2')
            capImages = ImgCapture()
            StreamProps.set_Capture(StreamProps, capImages)
            StreamProps.set_Quality(StreamProps, 90)
            server = ps.Streamer(address, StreamProps)
            print('Server started at','http://'+address[0]+':'+str(address[1]))
            server.serve_forever()
            while True:
                cv2.waitKey(1)
        
finally: 
    # Stop streaming
    pipeline.stop() 

Warning! No module named 'sounddevice'
Warning! No module named 'matplotlib'
Warning! No module named 'keras'
device_product_line D400
Server started at http://172.17.0.2:9001


192.168.1.74 - - [26/Apr/2023 04:52:30] "GET /index.html HTTP/1.1" 200 -
192.168.1.74 - - [26/Apr/2023 04:52:31] "GET /stream.mjpg HTTP/1.1" 200 -
192.168.1.74 - - [26/Apr/2023 04:52:42] "GET /index.html HTTP/1.1" 200 -
192.168.1.74 - - [26/Apr/2023 04:52:42] "GET /stream.mjpg HTTP/1.1" 200 -


KeyboardInterrupt: 

In [ ]:
#cv2.destroyAllWindows()

In [ ]:
def template_demo():
    tpl = cv.imread("./files/images/match2.png")
    target = cv.imread("./files/images/match1.png")

    methods = [cv.TM_SQDIFF_NORMED, cv.TM_CCORR_NORMED, cv.TM_CCOEFF_NORMED]  # Only test these three template matching methods
    th, tw = tpl.shape[:2] #Take length and width

    for md in methods:
        print(md)
        result = cv.matchTemplate(target, tpl, md)  # Get matching results
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
        if md == cv.TM_SQDIFF_NORMED:  # cv.TM_SQDIFF_NORMED is the most similar when it is the smallest, most similar when the others are the largest
            tl = min_loc
        else:
            tl = max_loc

        br = (tl[0] + tw, tl[1] + th)
        cv.rectangle(target, tl, br, (0, 0, 255), 2)  # tl is the coordinates of the upper left corner, br is the coordinates of the lower right corner, so as to draw a rectangle
        cv.imshow("match-"+np.str(md), target)

In [9]:
def template_demo_more():
    img_rgb = cv.imread('./files/images/temp1.jpg')
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    template = cv.imread('./files/images/temp2.jpg', 0)
    h, w = template.shape[:2]

    res = cv.matchTemplate(img_gray, template, cv.TM_CCOEFF_NORMED)
    threshold = 0.8

    loc = np.where(res >= threshold)  # Coordinates y, x where the matching degree is greater than %80
    print(loc)
    for pt in zip(*loc[::-1]):  # * Indicates optional parameters
        right_bottom = (pt[0] + w, pt[1] + h)
        cv.rectangle(img_rgb, pt, right_bottom, (0, 0, 255), 2)
        cv.imshow("more",img_rgb)
        